In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### State

In [2]:
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]

### 상담사 chatbot

In [ ]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.messages import BaseMessage


def call_chatbot(messages: List[BaseMessage]) -> dict:
    print('==================================================================================================')
    print(f'상담사 chatbot')

    print(f'[고객 질문]: ')
    print(messages)
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                'You are a customer support agent for an airline. Answer in Korean.'
            ),
            MessagesPlaceholder(variable_name='messages') 
        ]        
    )

    llm = ChatOpenAI(
        api_key=key, 
        model='gpt-4o-mini', 
        temperature=0.6
    )

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"messages": messages})
    
    print(f"상담사 chatbot 답변: ")
    print(answer)
    
    print('==================================================================================================\n')

    return answer

In [4]:
messages = [('user', '안녕하세요?')]
call_chatbot(messages)

상담사 chatbot
[고객 질문]: 
[('user', '안녕하세요?')]
상담사 chatbot 답변: 
안녕하세요! 어떻게 도와드릴까요?



'안녕하세요! 어떻게 도와드릴까요?'

In [5]:
messages = [HumanMessage(content='안녕하세요?')]
call_chatbot(messages)

상담사 chatbot
[고객 질문]: 
[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={})]
상담사 chatbot 답변: 
안녕하세요! 무엇을 도와드릴까요?



'안녕하세요! 무엇을 도와드릴까요?'

### 메시지의 역할 교체

In [6]:
def _swap_roles(messages):          #  메시지 타입을 AI -> Human, Human -> AI 로 교체
    
    new_messages = []

    for m in messages:
        if isinstance(m, AIMessage):            
            # AIMessage 인 경우, HumanMessage 로 변환합니다.            
            new_messages.append(HumanMessage(content=m.content))
        else:                                   
            # HumanMessage 인 경우, AIMessage 로 변환합니다.            
            new_messages.append(AIMessage(content=m.content))

    return new_messages

In [7]:
message1 = [HumanMessage(content='교환을 하고 싶습니다.', id='1')]
result = _swap_roles(message1)
result

[AIMessage(content='교환을 하고 싶습니다.', additional_kwargs={}, response_metadata={})]

In [8]:
message2 = [AIMessage(content='필요한것이 있으신가요?', id='2')]
result2 = _swap_roles(message2)
result2

[HumanMessage(content='필요한것이 있으신가요?', additional_kwargs={}, response_metadata={})]

### 상담사 노드

In [93]:
def ai_assistant_node(state: State):
    print('==================================================================================================')
    print(f'🔄 [Node] AI 상담사 ai_assistant_node 🔄')

    # print(f"고객 질문: ")
    # print(state["messages"])

    print('--------------------')
    print(f"상담사 chatbot 노드를 호출합니다. ")
    print('--------------------')
    
    ai_response = call_chatbot(state["messages"])       # 상담사 챗봇 호출
   
    # print(f"상담사 답변: ")
    # print(ai_response)

    print('==================================================================================================\n')

    return {"messages": [("assistant", ai_response)]}

In [ ]:
state = State(messages=[HumanMessage(content='교환을 하고 싶습니다.', id='1')])
result3 = ai_assistant_node(state)
result3

🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
상담사 chatbot
[고객 질문]: 
[HumanMessage(content='교환을 하고 싶습니다.', additional_kwargs={}, response_metadata={}, id='1')]


### 고객이 사용할 시나리오

In [ ]:
def create_scenario(name: str, instructions: str):
    system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

Your name is {name}.

# Instructions:
{instructions}

[IMPORTANT] 
- When you are finished with the conversation, respond with a single word 'FINISHED'
- You must speak in Korean."""


    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="messages")
        ]
    )

    prompt = prompt.partial(name=name, instructions=instructions)

    return prompt

In [ ]:
instructions = """You are trying to get a refund for the trip you took to Jeju Island. \
You want them to give you ALL the money back. This trip happened last year."""

name = '홍길동'

# create_scenario(name, instructions).pretty_print()

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0.6
)

simulated_user = create_scenario(name, instructions) | llm | StrOutputParser()

### 사용자 노드

In [ ]:
def simulated_user_node(state: State):

    print('==================================================================================================')
    print(f'🔄 [Node] 고객 노드 simulated_user_node 🔄')
    
    new_messages = _swap_roles(state["messages"])

    # print(f"사용자 질문: ")
    # print(new_messages)

    response = simulated_user.invoke({'messages': new_messages})

    # print(f"답변: ")
    # print(response)
    print('==================================================================================================\n')

    return {'messages': [('user', response)]}

In [ ]:
inputs = {
    "messages": [AIMessage(content="무엇을 도와드릴까요?")]
    
}

simulated_user_node(inputs)

🔄 [Node] 고객 노드 simulated_user_node 🔄



{'messages': [('user',
   '안녕하세요. 저는 작년에 제주도로 여행을 갔었는데, 그 여행에 대한 환불을 요청하고 싶습니다. 모든 금액을 환불받고 싶습니다. 도와주실 수 있나요?')]}

In [ ]:
def should_continue(state: State):
    print('\n==================================================================================================')
    print(f'===== [계속 실행] should_continue() =====')

    if len(state["messages"]) > 6:
        print('end')
        return "end"
    elif state["messages"][-1].content == "FINISHED":
        print('end')
        return "end"
    else:
        print('continue')
        return "continue"

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_teddynote.graphs import visualize_graph
from langgraph.graph import StateGraph, END

In [ ]:
######### 1. 그래프 생성 #########

graph_builder = StateGraph(State)

In [ ]:
######### 2. 노드 정의  #########

graph_builder.add_node('simulated_user', simulated_user_node)
graph_builder.add_node('ai_assistant', ai_assistant_node)

In [ ]:
######### 3. 그래프 엣지(Edge) 추가  #########

graph_builder.add_edge('ai_assistant', 'simulated_user')

In [ ]:
graph_builder.add_conditional_edges(
    "simulated_user",
    should_continue,
    {
        "end": END,                     
        "continue": "ai_assistant", 
    },
)

In [ ]:
######### 4. 그래프 진입점  #########

graph_builder.set_entry_point('ai_assistant')

In [ ]:
######### 5. 그래프 컴파일(compile)  #########

simulation = graph_builder.compile()

In [ ]:
visualize_graph(simulation)

[ERROR] Visualize Graph Error: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)


In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 입력 메시지 설정
inputs = {
    "messages": [HumanMessage(content="안녕하세요? 저 지금 좀 화가 많이 났습니다^^")]
}


for event in simulation.stream(inputs, config=config):
    for node_name, node_chunk in event.items():
        print('[node_name]:', node_name)
        print('[node_chunk]:\n', node_chunk)

🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
🔄 [Node] 상담사 chatbot 노드 🔄
[고객 질문]: 
[HumanMessage(content='안녕하세요? 저 지금 좀 화가 많이 났습니다^^', additional_kwargs={}, response_metadata={}, id='57ac1c3e-a204-4d35-90a3-c6f47b9b1605')]
상담사 chatbot 답변: 
안녕하세요! 고객님, 어떤 문제로 인해 화가 나신 건지 말씀해 주시면 최선을 다해 도와드리겠습니다. 어떤 부분에서 불편을 겪으셨는지 알려주실 수 있을까요?


[node_name]: ai_assistant
[node_chunk]:
 {'messages': [('assistant', '안녕하세요! 고객님, 어떤 문제로 인해 화가 나신 건지 말씀해 주시면 최선을 다해 도와드리겠습니다. 어떤 부분에서 불편을 겪으셨는지 알려주실 수 있을까요?')]}
🔄 [Node] 고객 노드 simulated_user_node 🔄


===== [계속 실행] should_continue() =====
continue
[node_name]: simulated_user
[node_chunk]:
 {'messages': [('user', '작년 제주도로 여행을 갔었는데, 그 여행에 대한 환불을 받고 싶습니다. 전액 환불이 필요합니다.')]}
🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
🔄 [Node] 상담사 chatbot 노드 🔄
[고객 질문]: 
[HumanMessage(content='안녕하세요? 저 지금 좀 화가 많이 났습니다^^', additional_kwargs={}, response_metadata={}, id='57ac1c

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 입력 메시지 설정
inputs = {
    "messages": [HumanMessage(content="안녕하세요? 저 지금 좀 화가 많이 났습니다^^")]
}


for event in simulation.stream(inputs, config=config, stream_mode='updates'):
    for node_name, node_chunk in event.items():
        print('[node_name]:', node_name)
        print('[node_chunk]:\n', node_chunk)

🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
🔄 [Node] 상담사 chatbot 노드 🔄
[고객 질문]: 
[HumanMessage(content='안녕하세요? 저 지금 좀 화가 많이 났습니다^^', additional_kwargs={}, response_metadata={}, id='0011bb93-e93e-4b83-ae9e-c9a43b86a3b6')]
상담사 chatbot 답변: 
안녕하세요! 고객님, 불편을 드려서 정말 죄송합니다. 어떤 문제로 화가 나셨는지 말씀해 주시면, 최대한 도와드리겠습니다.


[node_name]: ai_assistant
[node_chunk]:
 {'messages': [('assistant', '안녕하세요! 고객님, 불편을 드려서 정말 죄송합니다. 어떤 문제로 화가 나셨는지 말씀해 주시면, 최대한 도와드리겠습니다.')]}
🔄 [Node] 고객 노드 simulated_user_node 🔄


===== [계속 실행] should_continue() =====
continue
[node_name]: simulated_user
[node_chunk]:
 {'messages': [('user', '작년에 제주도로 여행을 갔었는데, 그 여행에 대한 환불을 요청하고 싶습니다. 모든 돈을 돌려받고 싶습니다.')]}
🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
🔄 [Node] 상담사 chatbot 노드 🔄
[고객 질문]: 
[HumanMessage(content='안녕하세요? 저 지금 좀 화가 많이 났습니다^^', additional_kwargs={}, response_metadata={}, id='0011bb93-e93e-4b83-ae9e-c9a43b86a3b6')

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 입력 메시지 설정
inputs = {
    "messages": [HumanMessage(content="안녕하세요? 저 지금 좀 화가 많이 났습니다^^")]
}

i = 1

for chunk in simulation.stream(inputs, config=config, stream_mode="updates"):
    print(f"===== for 시작 {i} =====")

    j = 1


    for state_key, state_value in chunk.items():
        print(f"== for 시작 {i} - {j} ==")
        print(f"state_key : {state_key }")
        print(f"state_value: \n {state_value}")
        print(f"== for  끝 {i} - {j}==")

        j=j+1
    
    print(f"===== for 끝 {i} =====")
    i=i+1

🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
🔄 [Node] 상담사 chatbot 노드 🔄
[고객 질문]: 
[HumanMessage(content='안녕하세요? 저 지금 좀 화가 많이 났습니다^^', additional_kwargs={}, response_metadata={}, id='32171272-420f-402c-94eb-84e39254e2ef')]
상담사 chatbot 답변: 
안녕하세요! 고객님, 화가 나신 것에 대해 정말 죄송합니다. 어떤 문제가 있으신지 말씀해 주시면 최선을 다해 도와드리겠습니다.


===== for 시작 1 =====
== for 시작 1 - 1 ==
state_key : ai_assistant
state_value: 
 {'messages': [('assistant', '안녕하세요! 고객님, 화가 나신 것에 대해 정말 죄송합니다. 어떤 문제가 있으신지 말씀해 주시면 최선을 다해 도와드리겠습니다.')]}
== for  끝 1 - 1==
===== for 끝 1 =====
🔄 [Node] 고객 노드 simulated_user_node 🔄


===== [계속 실행] should_continue() =====
continue
===== for 시작 2 =====
== for 시작 2 - 1 ==
state_key : simulated_user
state_value: 
 {'messages': [('user', '작년 제주도 여행에 대한 환불을 요청하고 싶습니다. 제가 지불한 금액을 전부 돌려받고 싶어요.')]}
== for  끝 2 - 1==
===== for 끝 2 =====
🔄 [Node] AI 상담사 ai_assistant_node 🔄
--------------------
상담사 chatbot 노드를 호출합니다. 
--------------------
🔄 [Node] 상담사 chatbot 